각 단어로 구성된 입력값은 임베딩된 벡터로 변형된다. 이후 각 벡터를 평균해서 하나의 벡터로 만든다.     
이후에 하나의 은닉층을 거친 후 하나의 결괏값을 뽑는 구조다.     
마지막으로 나온 결괏깞에 시그모이드 함수를 적용해 0과 1 사이의 값을 구한다.    

In [10]:
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras import layers

In [11]:
samples = [
    '너 오늘 이뻐 보인다',
    '나는 오늘 기분이 더러워',
    '끝내주는데, 좋은 일이 있나봐',
    '나 좋은 일이 생겼어',
    '아 오늘 진짜 짜증나',
    '환상적인데, 정말 좋은거 같아'
]
labels = [[1], [0], [1], [1], [0], [1]]

In [12]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
word_index = tokenizer.word_index

In [13]:
sequences

[[4, 1, 5, 6],
 [7, 1, 8, 9],
 [10, 2, 3, 11],
 [12, 2, 3, 13],
 [14, 1, 15, 16],
 [17, 18, 19, 20]]

In [14]:
word_index

{'오늘': 1,
 '좋은': 2,
 '일이': 3,
 '너': 4,
 '이뻐': 5,
 '보인다': 6,
 '나는': 7,
 '기분이': 8,
 '더러워': 9,
 '끝내주는데': 10,
 '있나봐': 11,
 '나': 12,
 '생겼어': 13,
 '아': 14,
 '진짜': 15,
 '짜증나': 16,
 '환상적인데': 17,
 '정말': 18,
 '좋은거': 19,
 '같아': 20}

In [15]:
# 필요한 변수를 정의
batch_size = 2
num_epochs = 100
vocab_size = len(word_index) + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

In [16]:
# Sequential 객체를 생성한 후 각 층을 추가한다.
model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, emb_size, input_length = 4)) # 입력값을 임베딩
model.add(layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1))) # 각 단어 벡터를 평균
model.add(layers.Dense(hidden_dimension, activation='relu')) 
model.add(layers.Dense(output_dimension, activation='sigmoid'))

In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
            loss='binary_crossentropy',
            metrics=['accuracy'])

In [ ]:
model.fit(sequences, labels, epochs=num_epochs, batch_size=batch_size)

Functional API, Subclassing 방법으로 동일한 모델을 구현해보고 학습

In [ ]:
inputs = layers.Input(shape = (4, ))
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis=1)
hidden_layer = layers.Dense(hidden_dimension, activation='relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation='sigmoid')(hidden_layer)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizer.Adam(0.001),
            loss='binary_crossentropy',
            metrircs=['accuracy'])
model.fit(sequences, labels, epochs=num_epochs, batch_size=batch_size)

In [ ]:
class CustomModel(tf.keras.Model):
    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name='my_model')
        self.embedding = layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation='relu')
        self.output_layer = layers.Dense(output_dimension, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis = 1)
        x = self.dense_layer(x)
        x = self.output_layer(x)

        return x 
    
model = CustomModel(vocab_size=vocab_size,
                    embed_dimension=emb_size,
                    hidden_dimension=hidden_dimension,
                    output_dimension=output_dimension)
model.compile(optimizer=tf.keras.Adam(0.001),
            loss='binary_crossentropy',
            metrics=['accuracy'])
model.fit(sequences, labels, epochs=num_epochs, batch_size=batch_size)
